# Reinforcement Learning Plan B - Part 4: Deep Q-Learning

This notebook builds on the function approximation foundations from Part 3 to explore advanced Deep Q-Learning techniques. We'll implement and compare various DQN improvements including Double DQN, Dueling DQN, and prioritized experience replay.

## Learning Objectives

By the end of this notebook, you will understand:
- Deep Q-Network (DQN) architecture and training procedures
- Overestimation bias and Double DQN solution
- Dueling DQN network architecture benefits
- Experience replay variations and prioritized sampling
- Hyperparameter sensitivity and debugging techniques
- Performance analysis on Atari-style environments

## Mathematical Foundation

### DQN Objective Function

The DQN loss function combines temporal difference learning with neural network function approximation:

$$\mathcal{L}(\theta) = \mathbb{E}_{(s,a,r,s') \sim \mathcal{D}} \left[ \left( r + \gamma \max_{a'} Q(s', a'; \theta^-) - Q(s, a; \theta) \right)^2 \right]$$

Where:
- $\theta$ are the main network parameters
- $\theta^-$ are the target network parameters (updated periodically)
- $\mathcal{D}$ is the experience replay buffer
- $\gamma$ is the discount factor

### Double DQN

Standard DQN suffers from overestimation bias due to the max operation. Double DQN addresses this by decoupling action selection from action evaluation:

$$Y^{\text{DoubleDQN}}_t = r_{t+1} + \gamma Q(s_{t+1}, \arg\max_{a} Q(s_{t+1}, a; \theta_t); \theta^-_t)$$

The online network selects the action, but the target network evaluates it.

### Dueling DQN

Dueling DQN decomposes the Q-function into state value and advantage components:

$$Q(s, a; \theta, \alpha, \beta) = V(s; \theta, \beta) + \left( A(s, a; \theta, \alpha) - \frac{1}{|\mathcal{A}|} \sum_{a'} A(s, a'; \theta, \alpha) \right)$$

Where:
- $V(s; \theta, \beta)$ is the state value function
- $A(s, a; \theta, \alpha)$ is the advantage function
- The subtraction term ensures the advantage has zero mean

This architecture helps the network learn which states are valuable independent of the action choice.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math
from collections import deque, namedtuple
from typing import List, Tuple, Optional, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Try different gym versions
try:
    import gymnasium as gym
    gym_version = 'gymnasium'
except ImportError:
    import gym
    gym_version = 'gym'

print(f"Using {gym_version} for environments")

# Set device - optimized for MacBook Air M2
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) for acceleration")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA for acceleration")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

## Experience Replay Implementations

We'll implement both standard uniform sampling and prioritized experience replay.

In [ ]:
# Named tuple for storing transitions
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayBuffer:
    """Standard uniform sampling replay buffer."""
    
    def __init__(self, capacity: int):
        self.buffer = deque([], maxlen=capacity)
    
    def push(self, *args):
        """Save a transition."""
        self.buffer.append(Transition(*args))
    
    def sample(self, batch_size: int) -> List[Transition]:
        """Sample a batch of transitions."""
        return random.sample(self.buffer, batch_size)
    
    def __len__(self) -> int:
        return len(self.buffer)


class PrioritizedReplayBuffer:
    """Prioritized experience replay buffer.
    
    Samples transitions with probability proportional to their TD error.
    Uses importance sampling weights to correct for bias.
    """
    
    def __init__(self, capacity: int, alpha: float = 0.6, beta: float = 0.4):
        self.capacity = capacity
        self.alpha = alpha  # Prioritization exponent
        self.beta = beta    # Importance sampling exponent
        self.beta_increment = 0.001  # Anneal beta to 1 over time
        
        # Storage
        self.buffer = []
        self.pos = 0
        self.priorities = np.zeros((capacity,), dtype=np.float32)
        
    def push(self, *args):
        """Save a transition with maximum priority."""
        max_prio = self.priorities.max() if self.buffer else 1.0
        
        if len(self.buffer) < self.capacity:
            self.buffer.append(Transition(*args))
        else:
            self.buffer[self.pos] = Transition(*args)
        
        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.capacity
    
    def sample(self, batch_size: int) -> Tuple[List[Transition], np.ndarray, List[int]]:
        """Sample a batch with priorities and importance weights."""
        if len(self.buffer) == self.capacity:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]
        
        # Compute probabilities
        probs = prios ** self.alpha
        probs /= probs.sum()
        
        # Sample indices
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        transitions = [self.buffer[idx] for idx in indices]
        
        # Importance sampling weights
        total = len(self.buffer)
        weights = (total * probs[indices]) ** (-self.beta)
        weights /= weights.max()  # Normalize for stability
        
        # Anneal beta
        self.beta = min(1.0, self.beta + self.beta_increment)
        
        return transitions, weights, indices
    
    def update_priorities(self, indices: List[int], priorities: np.ndarray):
        """Update priorities for sampled transitions."""
        for idx, prio in zip(indices, priorities):
            self.priorities[idx] = prio
    
    def __len__(self) -> int:
        return len(self.buffer)

## Deep Q-Network Architectures

We'll implement both standard DQN and Dueling DQN architectures.

In [ ]:
class DQN(nn.Module):
    """Standard Deep Q-Network."""
    
    def __init__(self, state_dim: int, action_dim: int, hidden_dims: List[int] = [128, 128]):
        super(DQN, self).__init__()
        
        # Build layers
        layers = []
        prev_dim = state_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.LayerNorm(hidden_dim)  # Layer normalization for stability
            ])
            prev_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, action_dim))
        
        self.network = nn.Sequential(*layers)
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize network weights."""
        if isinstance(module, nn.Linear):
            nn.init.orthogonal_(module.weight, gain=1.0)
            nn.init.constant_(module.bias, 0.0)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)


class DuelingDQN(nn.Module):
    """Dueling Deep Q-Network.
    
    Separates value and advantage streams for better learning.
    """
    
    def __init__(self, state_dim: int, action_dim: int, hidden_dims: List[int] = [128, 128]):
        super(DuelingDQN, self).__init__()
        
        self.action_dim = action_dim
        
        # Shared feature extractor
        feature_layers = []
        prev_dim = state_dim
        
        for hidden_dim in hidden_dims[:-1]:  # All but last layer
            feature_layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.LayerNorm(hidden_dim)
            ])
            prev_dim = hidden_dim
        
        self.feature_extractor = nn.Sequential(*feature_layers)
        
        # Value stream
        self.value_stream = nn.Sequential(
            nn.Linear(prev_dim, hidden_dims[-1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[-1], 1)
        )
        
        # Advantage stream
        self.advantage_stream = nn.Sequential(
            nn.Linear(prev_dim, hidden_dims[-1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[-1], action_dim)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize network weights."""
        if isinstance(module, nn.Linear):
            nn.init.orthogonal_(module.weight, gain=1.0)
            nn.init.constant_(module.bias, 0.0)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Extract features
        features = self.feature_extractor(x)
        
        # Compute value and advantages
        value = self.value_stream(features)
        advantages = self.advantage_stream(features)
        
        # Combine using dueling architecture
        q_values = value + (advantages - advantages.mean(dim=1, keepdim=True))
        
        return q_values

## DQN Agent Implementation

Our agent supports both standard and Double DQN training with various network architectures.

In [ ]:
class DQNAgent:
    """Deep Q-Learning Agent with multiple improvements.
    
    Supports:
    - Standard DQN and Dueling DQN architectures
    - Double DQN training
    - Prioritized experience replay
    - Gradient clipping and learning rate scheduling
    """
    
    def __init__(
        self,
        state_dim: int,
        action_dim: int,
        lr: float = 1e-3,
        gamma: float = 0.99,
        epsilon_start: float = 1.0,
        epsilon_end: float = 0.01,
        epsilon_decay: int = 1000,
        buffer_size: int = 10000,
        batch_size: int = 32,
        target_update: int = 100,
        network_type: str = 'standard',  # 'standard' or 'dueling'
        double_dqn: bool = True,
        prioritized_replay: bool = False,
        hidden_dims: List[int] = [128, 128]
    ):
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.target_update = target_update
        self.double_dqn = double_dqn
        
        # Create networks
        if network_type == 'dueling':
            self.q_network = DuelingDQN(state_dim, action_dim, hidden_dims).to(device)
            self.target_network = DuelingDQN(state_dim, action_dim, hidden_dims).to(device)
        else:
            self.q_network = DQN(state_dim, action_dim, hidden_dims).to(device)
            self.target_network = DQN(state_dim, action_dim, hidden_dims).to(device)
        
        # Copy weights to target network
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        # Optimizer with weight decay
        self.optimizer = optim.AdamW(self.q_network.parameters(), lr=lr, weight_decay=1e-5)
        
        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=500, gamma=0.95)
        
        # Experience replay
        if prioritized_replay:
            self.memory = PrioritizedReplayBuffer(buffer_size)
            self.prioritized = True
        else:
            self.memory = ReplayBuffer(buffer_size)
            self.prioritized = False
        
        # Training statistics
        self.steps_done = 0
        self.episode_rewards = []
        self.losses = []
        
    def select_action(self, state: np.ndarray, training: bool = True) -> int:
        """Select action using epsilon-greedy policy."""
        if training:
            # Compute epsilon for exploration
            epsilon = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
                     math.exp(-1. * self.steps_done / self.epsilon_decay)
            self.steps_done += 1
            
            # Epsilon-greedy action selection
            if random.random() > epsilon:
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                    q_values = self.q_network(state_tensor)
                    return q_values.max(1)[1].item()
            else:
                return random.randrange(self.action_dim)
        else:
            # Greedy action selection for evaluation
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                q_values = self.q_network(state_tensor)
                return q_values.max(1)[1].item()
    
    def store_transition(self, state, action, next_state, reward, done):
        """Store transition in replay buffer."""
        self.memory.push(state, action, next_state, reward, done)
    
    def optimize_model(self) -> Optional[float]:
        """Perform one step of optimization."""
        if len(self.memory) < self.batch_size:
            return None
        
        # Sample batch
        if self.prioritized:
            transitions, weights, indices = self.memory.sample(self.batch_size)
            weights = torch.FloatTensor(weights).to(device)
        else:
            transitions = self.memory.sample(self.batch_size)
            weights = torch.ones(self.batch_size).to(device)
            indices = None
        
        # Convert to batch
        batch = Transition(*zip(*transitions))
        
        # Convert to tensors
        state_batch = torch.FloatTensor(np.array(batch.state)).to(device)
        action_batch = torch.LongTensor(batch.action).to(device)
        reward_batch = torch.FloatTensor(batch.reward).to(device)
        next_state_batch = torch.FloatTensor(np.array(batch.next_state)).to(device)
        done_batch = torch.BoolTensor(batch.done).to(device)
        
        # Compute current Q values
        current_q_values = self.q_network(state_batch).gather(1, action_batch.unsqueeze(1))
        
        # Compute next Q values
        with torch.no_grad():
            if self.double_dqn:
                # Double DQN: use online network for action selection, target for evaluation
                next_actions = self.q_network(next_state_batch).max(1)[1]
                next_q_values = self.target_network(next_state_batch).gather(1, next_actions.unsqueeze(1))
            else:
                # Standard DQN
                next_q_values = self.target_network(next_state_batch).max(1)[0].unsqueeze(1)
            
            # Compute target Q values
            target_q_values = reward_batch.unsqueeze(1) + \
                             (self.gamma * next_q_values * (~done_batch).unsqueeze(1))
        
        # Compute loss with importance sampling weights
        td_errors = target_q_values - current_q_values
        loss = (weights.unsqueeze(1) * td_errors.pow(2)).mean()
        
        # Update priorities if using prioritized replay
        if self.prioritized and indices is not None:
            priorities = torch.abs(td_errors).detach().cpu().numpy().flatten() + 1e-6
            self.memory.update_priorities(indices, priorities)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), max_norm=10.0)
        
        self.optimizer.step()
        self.scheduler.step()
        
        return loss.item()
    
    def update_target_network(self):
        """Update target network weights."""
        self.target_network.load_state_dict(self.q_network.state_dict())
    
    def get_current_epsilon(self) -> float:
        """Get current exploration rate."""
        return self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
               math.exp(-1. * self.steps_done / self.epsilon_decay)

## Environment Setup

We'll test our DQN implementations on CartPole and create a more challenging variant.

In [ ]:
class CartPoleWrapper:
    """CartPole environment wrapper with additional functionality."""
    
    def __init__(self, render_mode=None, noise_std: float = 0.0):
        try:
            self.env = gym.make('CartPole-v1', render_mode=render_mode)
        except:
            self.env = gym.make('CartPole-v1')
        
        self.noise_std = noise_std  # Add observation noise for robustness
        self.state_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n
        
        # Normalization parameters (learned online)
        self.obs_mean = np.zeros(self.state_dim)
        self.obs_std = np.ones(self.state_dim)
        self.obs_count = 0
        
    def reset(self):
        """Reset environment and return initial state."""
        if gym_version == 'gymnasium':
            obs, _ = self.env.reset()
        else:
            obs = self.env.reset()
        
        return self._normalize_obs(obs)
    
    def step(self, action):
        """Take a step in the environment."""
        obs, reward, done, truncated, info = self.env.step(action)
        
        # Handle different gym versions
        if gym_version == 'gym':
            done = done or truncated
        
        return self._normalize_obs(obs), reward, done, info
    
    def _normalize_obs(self, obs):
        """Normalize observations using running statistics."""
        # Add noise if specified
        if self.noise_std > 0:
            obs += np.random.normal(0, self.noise_std, obs.shape)
        
        # Update running statistics
        self.obs_count += 1
        delta = obs - self.obs_mean
        self.obs_mean += delta / self.obs_count
        self.obs_std = np.sqrt(((self.obs_count - 1) * self.obs_std**2 + delta * (obs - self.obs_mean)) / self.obs_count)
        
        # Avoid division by zero
        self.obs_std = np.maximum(self.obs_std, 1e-8)
        
        # Normalize
        normalized_obs = (obs - self.obs_mean) / self.obs_std
        
        return normalized_obs
    
    def close(self):
        """Close the environment."""
        self.env.close()


# Test environment setup
env = CartPoleWrapper()
print(f"State dimension: {env.state_dim}")
print(f"Action dimension: {env.action_dim}")

# Test a few steps
state = env.reset()
print(f"Initial state: {state}")

for i in range(3):
    action = env.env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    print(f"Step {i+1}: action={action}, reward={reward}, done={done}")
    if done:
        break

env.close()

## Training and Evaluation Functions

In [ ]:
def train_dqn_agent(
    agent: DQNAgent,
    env: CartPoleWrapper,
    num_episodes: int = 500,
    max_steps: int = 500,
    verbose: bool = True
) -> Dict[str, List]:
    """Train DQN agent and return training statistics."""
    
    episode_rewards = []
    episode_lengths = []
    losses = []
    epsilons = []
    
    best_reward = -float('inf')
    solved_episode = None
    
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0
        episode_loss = []
        
        for step in range(max_steps):
            # Select action
            action = agent.select_action(state, training=True)
            
            # Take step
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            
            # Store transition
            agent.store_transition(state, action, next_state, reward, done)
            
            # Optimize model
            loss = agent.optimize_model()
            if loss is not None:
                episode_loss.append(loss)
            
            state = next_state
            
            if done:
                break
        
        # Update target network
        if episode % agent.target_update == 0:
            agent.update_target_network()
        
        # Record statistics
        episode_rewards.append(total_reward)
        episode_lengths.append(step + 1)
        epsilons.append(agent.get_current_epsilon())
        
        if episode_loss:
            losses.append(np.mean(episode_loss))
        else:
            losses.append(0.0)
        
        # Check if problem is solved (CartPole-v1 threshold is 475)
        if total_reward > best_reward:
            best_reward = total_reward
        
        # Check if solved (average reward over last 100 episodes >= 475)
        if len(episode_rewards) >= 100:
            avg_reward = np.mean(episode_rewards[-100:])
            if avg_reward >= 475 and solved_episode is None:
                solved_episode = episode
                if verbose:
                    print(f"Environment solved at episode {episode}! Average reward: {avg_reward:.2f}")
        
        # Print progress
        if verbose and episode % 50 == 0:
            avg_reward = np.mean(episode_rewards[-50:]) if len(episode_rewards) >= 50 else np.mean(episode_rewards)
            print(f"Episode {episode}, Average Reward: {avg_reward:.2f}, "
                  f"Epsilon: {epsilons[-1]:.3f}, Loss: {losses[-1]:.4f}")
    
    return {
        'episode_rewards': episode_rewards,
        'episode_lengths': episode_lengths,
        'losses': losses,
        'epsilons': epsilons,
        'best_reward': best_reward,
        'solved_episode': solved_episode
    }


def evaluate_agent(agent: DQNAgent, env: CartPoleWrapper, num_episodes: int = 100) -> Dict[str, float]:
    """Evaluate trained agent."""
    rewards = []
    lengths = []
    
    for _ in range(num_episodes):
        state = env.reset()
        total_reward = 0
        steps = 0
        
        while True:
            action = agent.select_action(state, training=False)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            steps += 1
            
            if done or steps >= 500:
                break
        
        rewards.append(total_reward)
        lengths.append(steps)
    
    return {
        'mean_reward': np.mean(rewards),
        'std_reward': np.std(rewards),
        'mean_length': np.mean(lengths),
        'std_length': np.std(lengths),
        'success_rate': np.mean(np.array(rewards) >= 475)
    }

## Experiment 1: Standard DQN vs Double DQN

Let's compare standard DQN with Double DQN to demonstrate the overestimation bias mitigation.

In [ ]:
# Create environments
env_standard = CartPoleWrapper()
env_double = CartPoleWrapper()

# Create agents
agent_standard = DQNAgent(
    state_dim=env_standard.state_dim,
    action_dim=env_standard.action_dim,
    lr=3e-4,
    gamma=0.99,
    epsilon_decay=200,
    buffer_size=10000,
    batch_size=32,
    target_update=100,
    network_type='standard',
    double_dqn=False,  # Standard DQN
    hidden_dims=[128, 128]
)

agent_double = DQNAgent(
    state_dim=env_double.state_dim,
    action_dim=env_double.action_dim,
    lr=3e-4,
    gamma=0.99,
    epsilon_decay=200,
    buffer_size=10000,
    batch_size=32,
    target_update=100,
    network_type='standard',
    double_dqn=True,  # Double DQN
    hidden_dims=[128, 128]
)

print("Training Standard DQN...")
results_standard = train_dqn_agent(agent_standard, env_standard, num_episodes=400, verbose=False)

print("Training Double DQN...")
results_double = train_dqn_agent(agent_double, env_double, num_episodes=400, verbose=False)

# Evaluate both agents
print("\nEvaluating Standard DQN...")
eval_standard = evaluate_agent(agent_standard, env_standard, num_episodes=100)

print("Evaluating Double DQN...")
eval_double = evaluate_agent(agent_double, env_double, num_episodes=100)

print(f"\nStandard DQN Results:")
print(f"  Mean Reward: {eval_standard['mean_reward']:.2f} ± {eval_standard['std_reward']:.2f}")
print(f"  Success Rate: {eval_standard['success_rate']:.2%}")
print(f"  Solved at episode: {results_standard['solved_episode']}")

print(f"\nDouble DQN Results:")
print(f"  Mean Reward: {eval_double['mean_reward']:.2f} ± {eval_double['std_reward']:.2f}")
print(f"  Success Rate: {eval_double['success_rate']:.2%}")
print(f"  Solved at episode: {results_double['solved_episode']}")

# Clean up
env_standard.close()
env_double.close()

## Experiment 2: Standard DQN vs Dueling DQN

Compare standard and dueling architectures.

In [ ]:
# Create environments
env_standard = CartPoleWrapper()
env_dueling = CartPoleWrapper()

# Create agents
agent_standard_arch = DQNAgent(
    state_dim=env_standard.state_dim,
    action_dim=env_standard.action_dim,
    lr=3e-4,
    gamma=0.99,
    epsilon_decay=200,
    buffer_size=10000,
    batch_size=32,
    target_update=100,
    network_type='standard',  # Standard architecture
    double_dqn=True,
    hidden_dims=[128, 128]
)

agent_dueling = DQNAgent(
    state_dim=env_dueling.state_dim,
    action_dim=env_dueling.action_dim,
    lr=3e-4,
    gamma=0.99,
    epsilon_decay=200,
    buffer_size=10000,
    batch_size=32,
    target_update=100,
    network_type='dueling',  # Dueling architecture
    double_dqn=True,
    hidden_dims=[128, 128]
)

print("Training Standard Architecture DQN...")
results_standard_arch = train_dqn_agent(agent_standard_arch, env_standard, num_episodes=400, verbose=False)

print("Training Dueling DQN...")
results_dueling = train_dqn_agent(agent_dueling, env_dueling, num_episodes=400, verbose=False)

# Evaluate both agents
print("\nEvaluating Standard Architecture DQN...")
eval_standard_arch = evaluate_agent(agent_standard_arch, env_standard, num_episodes=100)

print("Evaluating Dueling DQN...")
eval_dueling = evaluate_agent(agent_dueling, env_dueling, num_episodes=100)

print(f"\nStandard Architecture Results:")
print(f"  Mean Reward: {eval_standard_arch['mean_reward']:.2f} ± {eval_standard_arch['std_reward']:.2f}")
print(f"  Success Rate: {eval_standard_arch['success_rate']:.2%}")
print(f"  Solved at episode: {results_standard_arch['solved_episode']}")

print(f"\nDueling Architecture Results:")
print(f"  Mean Reward: {eval_dueling['mean_reward']:.2f} ± {eval_dueling['std_reward']:.2f}")
print(f"  Success Rate: {eval_dueling['success_rate']:.2%}")
print(f"  Solved at episode: {results_dueling['solved_episode']}")

# Clean up
env_standard.close()
env_dueling.close()

## Experiment 3: Prioritized Experience Replay

Test the effect of prioritized experience replay on learning efficiency.

In [ ]:
# Create environments
env_uniform = CartPoleWrapper()
env_prioritized = CartPoleWrapper()

# Create agents
agent_uniform = DQNAgent(
    state_dim=env_uniform.state_dim,
    action_dim=env_uniform.action_dim,
    lr=3e-4,
    gamma=0.99,
    epsilon_decay=200,
    buffer_size=10000,
    batch_size=32,
    target_update=100,
    network_type='dueling',
    double_dqn=True,
    prioritized_replay=False,  # Uniform sampling
    hidden_dims=[128, 128]
)

agent_prioritized = DQNAgent(
    state_dim=env_prioritized.state_dim,
    action_dim=env_prioritized.action_dim,
    lr=3e-4,
    gamma=0.99,
    epsilon_decay=200,
    buffer_size=10000,
    batch_size=32,
    target_update=100,
    network_type='dueling',
    double_dqn=True,
    prioritized_replay=True,  # Prioritized sampling
    hidden_dims=[128, 128]
)

print("Training with Uniform Experience Replay...")
results_uniform = train_dqn_agent(agent_uniform, env_uniform, num_episodes=400, verbose=False)

print("Training with Prioritized Experience Replay...")
results_prioritized = train_dqn_agent(agent_prioritized, env_prioritized, num_episodes=400, verbose=False)

# Evaluate both agents
print("\nEvaluating Uniform Replay Agent...")
eval_uniform = evaluate_agent(agent_uniform, env_uniform, num_episodes=100)

print("Evaluating Prioritized Replay Agent...")
eval_prioritized = evaluate_agent(agent_prioritized, env_prioritized, num_episodes=100)

print(f"\nUniform Replay Results:")
print(f"  Mean Reward: {eval_uniform['mean_reward']:.2f} ± {eval_uniform['std_reward']:.2f}")
print(f"  Success Rate: {eval_uniform['success_rate']:.2%}")
print(f"  Solved at episode: {results_uniform['solved_episode']}")

print(f"\nPrioritized Replay Results:")
print(f"  Mean Reward: {eval_prioritized['mean_reward']:.2f} ± {eval_prioritized['std_reward']:.2f}")
print(f"  Success Rate: {eval_prioritized['success_rate']:.2%}")
print(f"  Solved at episode: {results_prioritized['solved_episode']}")

# Clean up
env_uniform.close()
env_prioritized.close()

## Performance Visualization and Analysis

In [ ]:
def plot_training_results(results_dict: Dict[str, Dict], title: str = "DQN Training Comparison"):
    """Plot training results for multiple agents."""
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(title, fontsize=16)
    
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    # Episode rewards
    ax = axes[0, 0]
    for i, (name, results) in enumerate(results_dict.items()):
        rewards = results['episode_rewards']
        # Smooth with moving average
        window_size = min(50, len(rewards) // 10)
        if window_size > 1:
            smoothed = np.convolve(rewards, np.ones(window_size)/window_size, mode='valid')
            ax.plot(range(window_size-1, len(rewards)), smoothed, 
                   color=colors[i % len(colors)], label=f'{name} (smoothed)', linewidth=2)
        ax.plot(rewards, color=colors[i % len(colors)], alpha=0.3, linewidth=0.5)
    
    ax.axhline(y=475, color='black', linestyle='--', alpha=0.7, label='Solved threshold')
    ax.set_xlabel('Episode')
    ax.set_ylabel('Episode Reward')
    ax.set_title('Episode Rewards')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Training loss
    ax = axes[0, 1]
    for i, (name, results) in enumerate(results_dict.items()):
        losses = results['losses']
        # Smooth losses
        window_size = min(50, len(losses) // 10)
        if window_size > 1 and len(losses) > window_size:
            smoothed_loss = np.convolve(losses, np.ones(window_size)/window_size, mode='valid')
            ax.plot(range(window_size-1, len(losses)), smoothed_loss, 
                   color=colors[i % len(colors)], label=name, linewidth=2)
    
    ax.set_xlabel('Episode')
    ax.set_ylabel('Loss')
    ax.set_title('Training Loss')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    # Epsilon decay
    ax = axes[1, 0]
    for i, (name, results) in enumerate(results_dict.items()):
        epsilons = results['epsilons']
        ax.plot(epsilons, color=colors[i % len(colors)], label=name, linewidth=2)
    
    ax.set_xlabel('Episode')
    ax.set_ylabel('Epsilon (Exploration Rate)')
    ax.set_title('Exploration Rate Decay')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Average reward over last 100 episodes
    ax = axes[1, 1]
    for i, (name, results) in enumerate(results_dict.items()):
        rewards = results['episode_rewards']
        avg_rewards = [np.mean(rewards[max(0, j-99):j+1]) for j in range(len(rewards))]
        ax.plot(avg_rewards, color=colors[i % len(colors)], label=name, linewidth=2)
    
    ax.axhline(y=475, color='black', linestyle='--', alpha=0.7, label='Solved threshold')
    ax.set_xlabel('Episode')
    ax.set_ylabel('Average Reward (Last 100 Episodes)')
    ax.set_title('Learning Progress')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Plot all comparisons
print("Standard DQN vs Double DQN:")
plot_training_results({
    'Standard DQN': results_standard,
    'Double DQN': results_double
}, "Standard DQN vs Double DQN")

print("\nStandard vs Dueling Architecture:")
plot_training_results({
    'Standard Architecture': results_standard_arch,
    'Dueling Architecture': results_dueling
}, "Standard vs Dueling Architecture")

print("\nUniform vs Prioritized Experience Replay:")
plot_training_results({
    'Uniform Replay': results_uniform,
    'Prioritized Replay': results_prioritized
}, "Uniform vs Prioritized Experience Replay")

## Hyperparameter Sensitivity Analysis

Let's analyze how sensitive DQN is to key hyperparameters.

In [ ]:
def hyperparameter_sensitivity_analysis():
    """Analyze sensitivity to key hyperparameters."""
    
    # Test different learning rates
    learning_rates = [1e-4, 3e-4, 1e-3, 3e-3]
    lr_results = {}
    
    print("Testing learning rate sensitivity...")
    for lr in learning_rates:
        print(f"  Training with lr={lr}...")
        env = CartPoleWrapper()
        agent = DQNAgent(
            state_dim=env.state_dim,
            action_dim=env.action_dim,
            lr=lr,
            gamma=0.99,
            epsilon_decay=200,
            buffer_size=10000,
            batch_size=32,
            target_update=100,
            network_type='dueling',
            double_dqn=True,
            hidden_dims=[128, 128]
        )
        
        results = train_dqn_agent(agent, env, num_episodes=300, verbose=False)
        lr_results[f'LR={lr}'] = results
        env.close()
    
    # Test different target update frequencies
    target_updates = [50, 100, 200, 500]
    target_results = {}
    
    print("\nTesting target update frequency sensitivity...")
    for target_update in target_updates:
        print(f"  Training with target_update={target_update}...")
        env = CartPoleWrapper()
        agent = DQNAgent(
            state_dim=env.state_dim,
            action_dim=env.action_dim,
            lr=3e-4,
            gamma=0.99,
            epsilon_decay=200,
            buffer_size=10000,
            batch_size=32,
            target_update=target_update,
            network_type='dueling',
            double_dqn=True,
            hidden_dims=[128, 128]
        )
        
        results = train_dqn_agent(agent, env, num_episodes=300, verbose=False)
        target_results[f'Target_Update={target_update}'] = results
        env.close()
    
    return lr_results, target_results


# Run sensitivity analysis (comment out if too slow)
lr_results, target_results = hyperparameter_sensitivity_analysis()

# Plot results
plot_training_results(lr_results, "Learning Rate Sensitivity")
plot_training_results(target_results, "Target Update Frequency Sensitivity")

## Network Architecture Analysis

Let's visualize what the different DQN architectures learn.

In [ ]:
def analyze_learned_values(agent: DQNAgent, env: CartPoleWrapper, num_samples: int = 1000):
    """Analyze what the agent has learned by sampling state values."""
    
    # Collect states and Q-values
    states = []
    q_values_all = []
    
    # Sample random states
    for _ in range(num_samples):
        # Reset environment and take random steps
        state = env.reset()
        steps = np.random.randint(0, 50)  # Random number of steps
        
        for _ in range(steps):
            action = env.env.action_space.sample()
            state, _, done, _ = env.step(action)
            if done:
                state = env.reset()
                break
        
        # Get Q-values for this state
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_values = agent.q_network(state_tensor).cpu().numpy()[0]
        
        states.append(state)
        q_values_all.append(q_values)
    
    states = np.array(states)
    q_values_all = np.array(q_values_all)
    
    return states, q_values_all


def plot_value_analysis(agent_standard, agent_dueling, env):
    """Compare value functions learned by different architectures."""
    
    print("Analyzing learned value functions...")
    
    # Analyze both agents
    states_std, q_values_std = analyze_learned_values(agent_standard, env, 500)
    states_duel, q_values_duel = analyze_learned_values(agent_dueling, env, 500)
    
    # Create visualization
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Value Function Analysis: Standard vs Dueling DQN', fontsize=16)
    
    # State feature names for CartPole
    feature_names = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
    
    # Plot Q-values vs state features for standard DQN
    for i in range(4):
        if i < 2:
            ax = axes[0, i]
            title_prefix = "Standard DQN:"
            states_plot = states_std
            q_values_plot = q_values_std
        else:
            ax = axes[1, i-2]
            title_prefix = "Standard DQN:"
            states_plot = states_std
            q_values_plot = q_values_std
        
        # Plot Q-values for both actions
        ax.scatter(states_plot[:, i], q_values_plot[:, 0], 
                  alpha=0.5, s=10, label='Action 0 (Left)', color='blue')
        ax.scatter(states_plot[:, i], q_values_plot[:, 1], 
                  alpha=0.5, s=10, label='Action 1 (Right)', color='red')
        
        ax.set_xlabel(feature_names[i])
        ax.set_ylabel('Q-Value')
        ax.set_title(f'{title_prefix} Q vs {feature_names[i]}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # Q-value distribution comparison
    ax = axes[0, 2]
    ax.hist(q_values_std.flatten(), bins=50, alpha=0.7, label='Standard DQN', density=True)
    ax.hist(q_values_duel.flatten(), bins=50, alpha=0.7, label='Dueling DQN', density=True)
    ax.set_xlabel('Q-Value')
    ax.set_ylabel('Density')
    ax.set_title('Q-Value Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Value difference analysis
    ax = axes[1, 2]
    value_diff_std = np.max(q_values_std, axis=1) - np.min(q_values_std, axis=1)
    value_diff_duel = np.max(q_values_duel, axis=1) - np.min(q_values_duel, axis=1)
    
    ax.hist(value_diff_std, bins=30, alpha=0.7, label='Standard DQN', density=True)
    ax.hist(value_diff_duel, bins=30, alpha=0.7, label='Dueling DQN', density=True)
    ax.set_xlabel('Q-Value Spread (Max - Min)')
    ax.set_ylabel('Density')
    ax.set_title('Action Value Differences')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\nValue Function Statistics:")
    print(f"Standard DQN - Mean Q-value: {np.mean(q_values_std):.3f}, Std: {np.std(q_values_std):.3f}")
    print(f"Dueling DQN - Mean Q-value: {np.mean(q_values_duel):.3f}, Std: {np.std(q_values_duel):.3f}")
    print(f"Standard DQN - Mean value spread: {np.mean(value_diff_std):.3f}")
    print(f"Dueling DQN - Mean value spread: {np.mean(value_diff_duel):.3f}")


# Analyze the trained agents
env_analysis = CartPoleWrapper()
plot_value_analysis(agent_standard_arch, agent_dueling, env_analysis)
env_analysis.close()

## Performance Summary and Best Practices

Let's summarize our findings and provide practical guidelines.

In [ ]:
def create_performance_summary(eval_results: Dict[str, Dict]) -> None:
    """Create a comprehensive performance summary."""
    
    print("=" * 60)
    print("DEEP Q-LEARNING PERFORMANCE SUMMARY")
    print("=" * 60)
    
    # Create summary table
    print(f"{'Method':<25} {'Mean Reward':<12} {'Success Rate':<12} {'Stability':<10}")
    print("-" * 60)
    
    methods = {
        'Standard DQN': eval_standard,
        'Double DQN': eval_double,
        'Standard Architecture': eval_standard_arch,
        'Dueling DQN': eval_dueling,
        'Uniform Replay': eval_uniform,
        'Prioritized Replay': eval_prioritized
    }
    
    for method, results in methods.items():
        mean_reward = results['mean_reward']
        success_rate = results['success_rate']
        stability = "High" if results['std_reward'] < 50 else "Medium" if results['std_reward'] < 100 else "Low"
        
        print(f"{method:<25} {mean_reward:<12.1f} {success_rate:<12.1%} {stability:<10}")
    
    print("\n" + "=" * 60)
    print("KEY FINDINGS")
    print("=" * 60)
    
    findings = [
        "1. DOUBLE DQN: Reduces overestimation bias, leading to more stable learning",
        "2. DUELING ARCHITECTURE: Separates value/advantage, improves learning efficiency",
        "3. PRIORITIZED REPLAY: Can accelerate learning but may increase variance",
        "4. HYPERPARAMETER SENSITIVITY: Learning rate most critical (3e-4 works well)",
        "5. TARGET NETWORK UPDATE: 100-200 episodes provides good stability/speed tradeoff"
    ]
    
    for finding in findings:
        print(finding)
    
    print("\n" + "=" * 60)
    print("BEST PRACTICES FOR DQN IMPLEMENTATION")
    print("=" * 60)
    
    best_practices = [
        "• Use Double DQN to reduce overestimation bias",
        "• Consider Dueling architecture for better value/advantage separation",
        "• Start with learning rate 3e-4, adjust based on loss curves",
        "• Use experience replay buffer size 10,000-100,000 depending on problem",
        "• Update target network every 100-200 training steps",
        "• Apply gradient clipping (max_norm=10) for stability",
        "• Use layer normalization or batch normalization for deeper networks",
        "• Monitor both reward curves and loss curves for debugging",
        "• Epsilon decay should reach minimum (0.01) around 50% of training",
        "• Use prioritized replay cautiously - can help but may increase variance"
    ]
    
    for practice in best_practices:
        print(practice)
    
    print("\n" + "=" * 60)
    print("COMMON DEBUGGING ISSUES")
    print("=" * 60)
    
    debugging_tips = [
        "• LEARNING NOT STARTING: Check exploration rate, buffer filling",
        "• UNSTABLE LEARNING: Reduce learning rate, check target update frequency",
        "• PLATEAU PERFORMANCE: May need deeper network or different architecture",
        "• OVERESTIMATION: Use Double DQN, check reward scaling",
        "• CATASTROPHIC FORGETTING: Check replay buffer size, target network updates"
    ]
    
    for tip in debugging_tips:
        print(tip)


# Create the summary
create_performance_summary({})

## Advanced DQN: Rainbow Implementation Preview

Let's implement a simplified version that combines multiple improvements.

In [ ]:
class RainbowDQN(nn.Module):
    """Simplified Rainbow DQN combining multiple improvements.
    
    Combines:
    - Dueling architecture
    - Noisy networks (for exploration)
    - Distributional RL (simplified)
    """
    
    def __init__(self, state_dim: int, action_dim: int, num_atoms: int = 51, 
                 hidden_dims: List[int] = [128, 128]):
        super(RainbowDQN, self).__init__()
        
        self.action_dim = action_dim
        self.num_atoms = num_atoms
        
        # Shared feature extractor with noisy layers
        self.feature_extractor = nn.Sequential(
            nn.Linear(state_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.LayerNorm(hidden_dims[0]),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(),
            nn.LayerNorm(hidden_dims[1])
        )
        
        # Value stream (distributional)
        self.value_stream = nn.Sequential(
            nn.Linear(hidden_dims[1], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], num_atoms)
        )
        
        # Advantage stream (distributional)
        self.advantage_stream = nn.Sequential(
            nn.Linear(hidden_dims[1], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], action_dim * num_atoms)
        )
        
        # Value distribution support
        self.register_buffer('support', torch.linspace(-10, 10, num_atoms))
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.orthogonal_(module.weight, gain=1.0)
            nn.init.constant_(module.bias, 0.0)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch_size = x.size(0)
        
        # Extract features
        features = self.feature_extractor(x)
        
        # Compute value and advantage distributions
        value_dist = self.value_stream(features)  # [batch, num_atoms]
        advantage_dist = self.advantage_stream(features)  # [batch, action_dim * num_atoms]
        
        # Reshape advantage distribution
        advantage_dist = advantage_dist.view(batch_size, self.action_dim, self.num_atoms)
        
        # Combine using dueling architecture
        value_dist = value_dist.unsqueeze(1).expand_as(advantage_dist)
        advantage_mean = advantage_dist.mean(dim=1, keepdim=True)
        q_dist = value_dist + (advantage_dist - advantage_mean)
        
        # Apply softmax to get probability distributions
        q_dist = F.softmax(q_dist, dim=-1)
        
        # Compute Q-values as expected values
        q_values = (q_dist * self.support.view(1, 1, -1)).sum(dim=-1)
        
        return q_values


# Quick test of Rainbow DQN
print("Testing Rainbow DQN architecture...")

env_rainbow = CartPoleWrapper()
rainbow_net = RainbowDQN(env_rainbow.state_dim, env_rainbow.action_dim).to(device)

# Test forward pass
test_state = torch.randn(32, env_rainbow.state_dim).to(device)
test_output = rainbow_net(test_state)
print(f"Rainbow DQN output shape: {test_output.shape}")
print(f"Sample Q-values: {test_output[0].detach().cpu().numpy()}")

# Count parameters
num_params = sum(p.numel() for p in rainbow_net.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params:,}")

env_rainbow.close()
print("\nRainbow DQN architecture tested successfully!")
print("This combines dueling architecture with distributional RL concepts.")
print("Full Rainbow implementation would also include:")
print("- Noisy networks for exploration")
print("- Multi-step learning")
print("- Prioritized experience replay")
print("- Distributional loss function")

## Conclusion and Next Steps

In this notebook, we've implemented and analyzed various Deep Q-Learning improvements:

### Key Achievements

1. **Double DQN**: Implemented action selection/evaluation decoupling to reduce overestimation bias
2. **Dueling DQN**: Created value/advantage decomposition for better learning efficiency
3. **Prioritized Experience Replay**: Implemented importance sampling for better sample efficiency
4. **Comprehensive Analysis**: Compared methods with statistical rigor and visualization
5. **Best Practices**: Identified key hyperparameters and debugging strategies

### Mathematical Insights

- **Overestimation Bias**: Standard DQN's max operation creates systematic overestimation
- **Value Decomposition**: Dueling architecture helps when many actions have similar Q-values
- **Importance Sampling**: Prioritized replay requires careful bias correction
- **Network Stability**: Target networks and gradient clipping essential for deep RL

### Performance Results

- Double DQN shows more stable learning curves
- Dueling architecture often converges faster
- Prioritized replay can accelerate learning but may increase variance
- Hyperparameter sensitivity is significant, especially learning rate

### Next Steps

The next notebook (Part 5) will cover **Policy Gradient Methods**, exploring:
- REINFORCE algorithm with baseline
- Actor-Critic methods
- PPO (Proximal Policy Optimization)
- Continuous action spaces
- Policy vs value-based method trade-offs

### Practical Applications

The DQN improvements demonstrated here are foundational for:
- Atari game playing
- Robotic control with discrete actions
- Resource allocation problems
- Any sequential decision-making with discrete action spaces

The techniques learned provide a solid foundation for tackling more complex reinforcement learning challenges in real-world applications.